This version uses Milvus through Docker Compose so you must have Docker installed to run this notebook (Milvus is spun up via `docker compose up -d` as shown in the block below)

In [66]:
# ! pip install pymilvus milvus langchain sentence-transformers tiktoken octoai-sdk
# docker compose up -d

In [67]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms.octoai_endpoint import OctoAIEndpoint

In [68]:
from dotenv import load_dotenv
import os

load_dotenv()

OCTOAI_API_TOKEN = os.environ.get("OCTOAI_API_TOKEN")

os.environ["OCTOAI_API_TOKEN"] = os.getenv("OCTOAI_API_TOKEN")

In [69]:
template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.\n Instruction:\n{question}\n Response: """
prompt = PromptTemplate.from_template(template)

In [70]:
llm = OctoAIEndpoint(
    endpoint_url="https://text.octoai.run/v1/chat/completions",
    model_kwargs={
        "model": "mixtral-8x7b-instruct-fp16",
        "max_tokens": 128,
        "presence_penalty": 0,
        "temperature": 0.01,
        "top_p": 0.9,
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant. Keep your responses limited to one short paragraph if possible.",
            },
        ],
    },
)

In [71]:
question = "Give me a tweet about elon musk"

llm_chain = LLMChain(prompt=prompt, llm=llm)

print(llm_chain.invoke(question)["text"])

 "Exciting to see the visionary leadership of Elon Musk, driving innovation in electric vehicles, space exploration, and sustainable energy. His work with Tesla, SpaceX, and SolarCity is truly inspiring. #ElonMusk #Innovation #FutureTech"


In [72]:
question = "What did I ask in the first prompt?"

llm_chain = LLMChain(prompt=prompt, llm=llm)

print(llm_chain.invoke(question)["text"])

 In the first prompt, you asked me to provide a brief introduction explaining that I am a helpful assistant and will keep responses limited to one short paragraph if possible.


In [73]:
from langchain_community.embeddings import OctoAIEmbeddings
from langchain_community.vectorstores import Milvus

In [74]:
embeddings = OctoAIEmbeddings(endpoint_url="https://text.octoai.run/v1/embeddings")

In [75]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
import os

In [80]:
files = os.listdir("./data")

In [81]:
file_texts = []

In [83]:
for file in files:
    if (file != ".DS_Store"):
        with open(f"./data/{file}") as f:
            file_text = f.read()

        # text_splitter = CharacterTextSplitter(
        #     separator="\n\n",
        #     chunk_size=1000,
        #     chunk_overlap=200,
        #     length_function=len,
        #     is_separator_regex=False,
        # )
        text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
            encoding_name="cl100k_base", chunk_size=1024, chunk_overlap=256, 
        )
        texts = text_splitter.split_text(file_text)
        for i, chunked_text in enumerate(texts):
            file_texts.append(Document(page_content=chunked_text, 
                metadata={"twitter_user": file.split(".")[0], "chunk_num": i}))

Created a chunk of size 1248, which is longer than the specified 1024
Created a chunk of size 1251, which is longer than the specified 1024
Created a chunk of size 1094, which is longer than the specified 1024
Created a chunk of size 2223, which is longer than the specified 1024
Created a chunk of size 1212, which is longer than the specified 1024
Created a chunk of size 1068, which is longer than the specified 1024
Created a chunk of size 2034, which is longer than the specified 1024
Created a chunk of size 1256, which is longer than the specified 1024
Created a chunk of size 1089, which is longer than the specified 1024
Created a chunk of size 1101, which is longer than the specified 1024
Created a chunk of size 1539, which is longer than the specified 1024
Created a chunk of size 1096, which is longer than the specified 1024
Created a chunk of size 5866, which is longer than the specified 1024
Created a chunk of size 1600, which is longer than the specified 1024


In [79]:
vector_store = Milvus.from_documents(
    file_texts,
    embedding=embeddings,
    connection_args={"host": "localhost", "port": 19530},
    collection_name="twitter_user"
)

ValueError: Error raised by the inference endpoint: Error: 400 {"message":"Invalid request.","errors":{"input.constrained-str":["String should have at most 10240 characters"],"input.list[constrained-str]":["Input should be a valid list"]}}(request_id: )

In [ ]:
file_texts[0]

Document(page_content="It all begins today! I will see you at 11:00 A.M. for the swearing-in. THE MOVEMENT CONTINUES - THE WORK BEGINS!\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\n Today we are not merely transferring power from one Administration to another\t or from one party to another ‚Äì but we are transferring...\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\n power from Washington\t D.C. and giving it back to you\t the American People. # InaugurationDay\t\t\t\t\t\t\t\t\t\t\t\t\t\t\n What truly matters is not which party controls our government\t but whether our government is controlled by the people.\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\n January 20th 2017\t will be remembered as the day the people became the rulers of this nation again.\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\n The forgotten men and women of our country will be forgotten no longer. From this moment on\t it‚Äôs going to be # AmericaFirst\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\n We will bring back our jobs. We will bring back our borders. We will bring back our wea

In [ ]:
retriever = vector_store.as_retriever()

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke("What policy does donald trump support")

' Based on the provided documents, there is no information about a specific policy that Donald Trump supports. However, the documents do mention that Donald Trump "failed America" according to Joe Biden, but the context of this statement is not explicitly linked to a particular policy.'

In [ ]:
# Let's make this a bit more fun and showcase the multilingual capabilities of Mixtal which really outshine other open source models

# Our Vector DB is populated with entries from english text - even the embedding model we're using here, GTE-Large
# works best on english text. However Mixtral has good mutlilingual capabilities in French, German, Spanish and Italian.
# So what we'll do is ask the assistant to only answer in french in the system and user prompt. RAG here is performed based on 
# english text, but upon producing the user response, the Mixtral LLM will generate tokens in a different language here (french)
french_llm = OctoAIEndpoint(
    endpoint_url="https://text.octoai.run/v1/chat/completions",
    model_kwargs={
        "model": "mixtral-8x7b-instruct-fp16",
        "max_tokens": 128,
        "presence_penalty": 0,
        "temperature": 0.1,
        "top_p": 0.9,
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant who responds in French and not in English.",
            },
        ],
    },
)

french_template = """Answer the question in French based only on the following context:
{context}

Question: {question}
"""
french_prompt = PromptTemplate.from_template(french_template)

In [ ]:
french_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | french_prompt
    | french_llm
    | StrOutputParser()
)

In [ ]:
fr_1 = french_chain.invoke("How big is the city of Seattle?")

In [ ]:
from pprint import pprint
pprint(fr_1)

(' La question ne fournit pas assez de contexte pour une réponse précise '
 'concernant la taille de la ville de Seattle. Cependant, je peux vous dire '
 "que Seattle est la plus grande ville de l'État de Washington aux États-Unis "
 "et qu'elle s'étend sur une superficie d'environ 217 km².")
